In [2]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=229abaff13519278c7a4f1be16c6ccc335659de70b98eeee74f141fa5b375491
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Suc

In [5]:
import cv2
from google.colab.patches import cv2_imshow
from roboflow import Roboflow
from pathlib import Path

In [6]:
# Constants
LABELED_RAW_IMAGES_PATH = Path('/content/drive/MyDrive/osteoporosis_data/labeled_data')
LABELED_ROI_IMAGES_PATH = Path('/content/drive/MyDrive/osteoporosis_data/labeled_rois')

### Load images

In [31]:
def get_images_paths(raw_data_path):
  file_list = [f for f in raw_data_path.glob('**/*') if f.is_file()]
  return file_list
  # # print("Original num of images:", len(file_list))
  # images = {}

  # for file_path in file_list:
  #   file_name = file_path.stem
  #   image = cv2.imread(str(file_path), cv2.IMREAD_GRAYSCALE)
  #   images[file_name] = image

  # return images

In [32]:
images_paths = get_images_paths(LABELED_RAW_IMAGES_PATH)
print("Number of images:", len(images_paths))

Number of images: 611


### Extract ROIs

In [33]:
# Knee detection model
rf = Roboflow(api_key="jeaBuRI3CrFlPbUFiwjn")
project = rf.workspace().project("knee-localization")
model_upper = project.version(1).model

rf = Roboflow(api_key="0Od18TnlofLc2tDnGyvg")
project = rf.workspace().project("knee-detector")
model_lower = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...


In [34]:
def validate_knee_roi_keys(knee_dict, knee_side):
  if knee_side in knee_dict:
    print(f"Duplicate key {knee_side}, removing knee side!!!!!!!!!!!!!!!!!!")
    return '?'
  return knee_side


def extract_bounding_box_coords(prediction):
    x1 = int(prediction['x'] - prediction['width'] / 2)
    x2 = int(prediction['x'] + prediction['width'] / 2)
    y1 = int(prediction['y'] - prediction['height'] / 2)
    y2 = int(prediction['y'] + prediction['height'] / 2)
    return (x1, x2, y1, y2)


def find_knee_bounding_boxes(image_path, model, confidence, overlap, max_rois):
  predict_result = model.predict(image_path, confidence=confidence, overlap=overlap).json()
  predictions = predict_result["predictions"]
  bounding_boxes_dict = {}

  # If there are move than two predictions, keep the top two according to the confidence score
  if len(predictions) > max_rois:
    print(f"Found more than {max_rois} predictions: {len(predictions)} removing unnecessary ones...")
    predictions.sort(key=lambda prediction: prediction["confidence"], reverse=True)
    predictions = predictions[:max_rois]

  for prediction in predictions:
    box = extract_bounding_box_coords(prediction)
    knee_side = validate_knee_roi_keys(bounding_boxes_dict, prediction["class"][0])
    bounding_boxes_dict[knee_side] = box

  return bounding_boxes_dict

In [35]:
def extract_knee_rois(images_paths, model_upper, model_lower):
  knee_rois_dict = {}

  for image_path in images_paths:
    raw_image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    image_name = image_path.stem

    if image_name[0].isupper():
      print("Extracting ROIs from:", image_name, " Using model upper")
      bounding_boxes_dict = find_knee_bounding_boxes(str(image_path), model_upper, confidence=50, overlap=50, max_rois=2)
    else:
      print("Extracting ROIs from:", image_name, " Using model lower")
      bounding_boxes_dict = find_knee_bounding_boxes(str(image_path), model_lower, confidence=50, overlap=50, max_rois=1)
    print(f"Found {len(bounding_boxes_dict)} ROIs")

    for knee_side, (x1, x2, y1, y2) in bounding_boxes_dict.items():
      knee_roi_image = raw_image[y1:y2, x1:x2]
      roi_key_name = f"{image_name}_{knee_side}"
      print("ROI name:", roi_key_name)
      knee_rois_dict[roi_key_name] = knee_roi_image
    print()


  return knee_rois_dict

In [36]:
knee_rois_dict = extract_knee_rois(images_paths, model_upper, model_lower)
print("Number of ROIs:", len(knee_rois_dict))

Extracting ROIs from: OS10  Using model upper
Found more than 2 predictions: 3 removing unnecessary ones...
Found 2 ROIs
ROI name: OS10_R
ROI name: OS10_L

Extracting ROIs from: OS12  Using model upper
Found 2 ROIs
ROI name: OS12_R
ROI name: OS12_L

Extracting ROIs from: OS11  Using model upper
Found 2 ROIs
ROI name: OS11_R
ROI name: OS11_L

Extracting ROIs from: OS1  Using model upper
Found 2 ROIs
ROI name: OS1_R
ROI name: OS1_L

Extracting ROIs from: OS19  Using model upper
Found 2 ROIs
ROI name: OS19_L
ROI name: OS19_R

Extracting ROIs from: OS17  Using model upper
Found 2 ROIs
ROI name: OS17_R
ROI name: OS17_L

Extracting ROIs from: OS14  Using model upper
Found 2 ROIs
ROI name: OS14_L
ROI name: OS14_R

Extracting ROIs from: OS3  Using model upper
Found 2 ROIs
ROI name: OS3_R
ROI name: OS3_L

Extracting ROIs from: OS22  Using model upper
Found 2 ROIs
ROI name: OS22_L
ROI name: OS22_R

Extracting ROIs from: OS31_s  Using model upper
Found 1 ROIs
ROI name: OS31_s_L

Extracting ROIs f

In [40]:
# downsampled_knee_rois_dict = {key: cv2.resize(value, (350, 400)) for key, value in knee_rois_dict.items()}
# knee_rois_dict["OS21_L"].shape
len(knee_rois_dict)

735

### Save images

In [41]:
for image_name, image in knee_rois_dict.items():
  file_path = f"{str(LABELED_ROI_IMAGES_PATH)}/{image_name}.png"

  if not cv2.imwrite(file_path, image):
      raise Exception("Failed saving", file_path)

In [42]:
len(knee_rois_dict)

735